In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.1 MB/s 
     |████████████████████████████████| 6.6 MB 59.5 MB/s 
     |████████████████████████████████| 86 kB 4.0 MB/s 
     |████████████████████████████████| 596 kB 93.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/AIFFELTHON/AIFFELTHON_BBC/no_nan_qna_set.csv', index_col=0)
# df = pd.read_table('/content/drive/MyDrive/AIFFELTHON/AIFFELTHON_BBC/moonee_qna_set.txt', sep=',', index_col=0)
# df.head()

In [ ]:
df['sub_type'].value_counts()

내비게이션      1058
타이어         675
시동          672
경고등         642
라이트         583
차량외부        389
차량내부        368
주행관련        348
하이패스        199
후방카메라       190
단말기         180
주유/충전카드     158
브레이크        142
블랙박스         90
주차장          48
위생문제         46
차량점검         13
사고조사         11
비치품          10
ADAS         10
충전기확인         4
Name: sub_type, dtype: int64

In [ ]:
sub_type_mapping = {
    '내비게이션': 0,
    '타이어': 1,
    '라이트': 2,
    '시동': 3,
    '경고등': 4,
    '차량외부': 5,
    '차량내부': 6,
    '주행관련': 7,
    '사고조사': 8,
    '단말기': 9,
    '주유/충전카드': 10,
    '후방카메라': 11,
    '하이패스': 12,
    '차량점검': 13,
    '브레이크': 14,
    '블랙박스': 15,
    '위생문제': 16,
    '주차장': 17,
    'ADAS': 18,
    '비치품': 19,
    '충전기확인': 20
}

func = lambda x: sub_type_mapping.get(x, x)

In [ ]:
df['label'] = df['sub_type'].map(func)

In [ ]:
# df.head()

> `stratify`: 분류 문제 다룰 때 중요한 파라미터
> - target값을 지정해주면 target의 class 비율을 유지한 채로 데이터셋을 split하게 됨


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df['label'].values, 
                                                  test_size=0.2, 
                                                  random_state=42, 
                                                  stratify=df['label'].values)

In [ ]:
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(4668,) (1168,) (4668,) (1168,)


In [ ]:
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [ ]:
df.groupby(['sub_type', 'label', 'data_type']).count()

inquiry  detail_type
sub_type label data_type                      
ADAS     18    train            8            8
               val              2            2
경고등      4     train          514          514
               val            128          128
내비게이션    0     train          846          846
               val            212          212
단말기      9     train          144          144
               val             36           36
라이트      2     train          466          466
               val            117          117
브레이크     14    train          114          114
               val             28           28
블랙박스     15    train           72           72
               val             18           18
비치품      19    train            8            8
               val              2            2
사고조사     8     train            9            9
               val              2            2
시동       3     train          538          538
               val            134          134
위생문제     16    train           37           37
               val              9            9
주유/충전카드  10    train          126          126
               val             32           32
주차장      17    train           38           38
               val             10           10
주행관련     7     train          278          278
               val             70           70
차량내부     6     train          294          294
               val             74           74
차량외부     5     train          311          311
               val             78           78
차량점검     13    train           11           11
               val              2            2
충전기확인    20    train            3            3
               val              1            1
타이어      1     train          540          540
               val            135          135
하이패스     12    train          159          159
               val             40           40
후방카메라    11    train          152          152
               val             38           38

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train']['inquiry'].values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val']['inquiry'].values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train']['label'].values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val']['label'].values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
len(dataset_train), len(dataset_val)

(4668, 1168)

In [ ]:
len(sub_type_mapping)

21

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased",
                                                      num_labels=len(sub_type_mapping),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in sub_type_mapping.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
import numpy as np
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'/content/drive/MyDrive/AIFFELTHON/AIFFELTHON_BBC/Following_bert/bert_model/bert-new/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased",
                                                      num_labels=len(sub_type_mapping),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/AIFFELTHON/AIFFELTHON_BBC/Following_bert/bert_model/bert-new/finetuned_BERT_epoch_1.model', map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [ ]:
accuracy_per_class(predictions, true_vals)

Class: 내비게이션
Accuracy: 194/212

Class: 타이어
Accuracy: 108/135

Class: 라이트
Accuracy: 77/117

Class: 시동
Accuracy: 116/134

Class: 경고등
Accuracy: 103/128

Class: 차량외부
Accuracy: 42/78

Class: 차량내부
Accuracy: 39/74

Class: 주행관련
Accuracy: 43/70

Class: 사고조사
Accuracy: 0/2

Class: 단말기
Accuracy: 4/36

Class: 주유/충전카드
Accuracy: 30/32

Class: 후방카메라
Accuracy: 30/38

Class: 하이패스
Accuracy: 37/40

Class: 차량점검
Accuracy: 0/2

Class: 브레이크
Accuracy: 21/28

Class: 블랙박스
Accuracy: 14/18

Class: 위생문제
Accuracy: 0/9

Class: 주차장
Accuracy: 3/10

Class: ADAS
Accuracy: 0/2

Class: 비치품
Accuracy: 0/2

Class: 충전기확인
Accuracy: 0/1



---

In [ ]:
model_path = '/content/drive/MyDrive/AIFFELTHON/AIFFELTHON_BBC/Following_bert/bert_model/bert-new/finetuned_BERT_epoch_1.model'

In [ ]:
from transformers import BertTokenizerFast
from transformers import TFBertForSequenceClassification
from transformers import TextClassificationPipeline

# 로드하기
# loaded_tokenizer = BertTokenizerFast.from_pretrained(model_path)
# loaded_model = TFBertForSequenceClassification.from_pretrained(model_path)

# loaded_tokenizer = BertTokenizer.from_pretrained(model_path)
# loaded_tokenizer = tokenizer

text_classifier = TextClassificationPipeline(
    tokenizer=tokenizer, 
    model=model, 
    framework='tf', 
    return_all_scores=True
)

'''
The model 'BertForSequenceClassification' is not supported for . Supported models are ['TFDebertaV2ForSequenceClassification', 'TFDebertaForSequenceClassification', 'TFRemBertForSequenceClassification', 'TFRoFormerForSequenceClassification', 'TFConvBertForSequenceClassification', 'TFDistilBertForSequenceClassification', 'TFAlbertForSequenceClassification', 'TFCamembertForSequenceClassification', 'TFXLMRobertaForSequenceClassification', 'TFLongformerForSequenceClassification', 'TFRobertaForSequenceClassification', 'TFLayoutLMForSequenceClassification', 'TFBertForSequenceClassification', 'TFXLNetForSequenceClassification', 'TFMobileBertForSequenceClassification', 'TFFlaubertForSequenceClassification', 'TFXLMForSequenceClassification', 'TFElectraForSequenceClassification', 'TFTapasForSequenceClassification', 'TFFunnelForSequenceClassification', 'TFGPT2ForSequenceClassification', 'TFGPTJForSequenceClassification', 'TFMPNetForSequenceClassification', 'TFOpenAIGPTForSequenceClassification', 'TFTransfoXLForSequenceClassification', 'TFCTRLForSequenceClassification'].
'''

The model 'BertForSequenceClassification' is not supported for . Supported models are ['TFDebertaV2ForSequenceClassification', 'TFDebertaForSequenceClassification', 'TFRemBertForSequenceClassification', 'TFRoFormerForSequenceClassification', 'TFConvBertForSequenceClassification', 'TFDistilBertForSequenceClassification', 'TFAlbertForSequenceClassification', 'TFCamembertForSequenceClassification', 'TFXLMRobertaForSequenceClassification', 'TFLongformerForSequenceClassification', 'TFRobertaForSequenceClassification', 'TFLayoutLMForSequenceClassification', 'TFBertForSequenceClassification', 'TFXLNetForSequenceClassification', 'TFMobileBertForSequenceClassification', 'TFFlaubertForSequenceClassification', 'TFXLMForSequenceClassification', 'TFElectraForSequenceClassification', 'TFTapasForSequenceClassification', 'TFFunnelForSequenceClassification', 'TFGPT2ForSequenceClassification', 'TFGPTJForSequenceClassification', 'TFMPNetForSequenceClassification', 'TFOpenAIGPTForSequenceClassification', 

"\nThe model 'BertForSequenceClassification' is not supported for . Supported models are ['TFDebertaV2ForSequenceClassification', 'TFDebertaForSequenceClassification', 'TFRemBertForSequenceClassification', 'TFRoFormerForSequenceClassification', 'TFConvBertForSequenceClassification', 'TFDistilBertForSequenceClassification', 'TFAlbertForSequenceClassification', 'TFCamembertForSequenceClassification', 'TFXLMRobertaForSequenceClassification', 'TFLongformerForSequenceClassification', 'TFRobertaForSequenceClassification', 'TFLayoutLMForSequenceClassification', 'TFBertForSequenceClassification', 'TFXLNetForSequenceClassification', 'TFMobileBertForSequenceClassification', 'TFFlaubertForSequenceClassification', 'TFXLMForSequenceClassification', 'TFElectraForSequenceClassification', 'TFTapasForSequenceClassification', 'TFFunnelForSequenceClassification', 'TFGPT2ForSequenceClassification', 'TFGPTJForSequenceClassification', 'TFMPNetForSequenceClassification', 'TFOpenAIGPTForSequenceClassification

In [ ]:
# 정렬 함수

# 맨 처음에 만든 sub_type_mapping 뒤집기 -> dic.get() 함수 쓰기 위해
sub_type_mapping_reverse = {v: k for k, v in sub_type_mapping.items()}

def get_category_name(sentence):
    rank = {}
    for i, n in enumerate(text_classifier(sentence)[0]): # 각 라벨 값, 확률값 dictionary로 새로 저장
        rank[i] = n['score']
    
    sorted_rank = dict(sorted(rank.items(), reverse=True, key=lambda x: x[1])) # dictionary 내림차순 정렬

    # sub_category만 보고 싶을 때
    # for key in sorted_rank.keys():
    #     print(sub_type_mapping_reverse.get(key)) # 확률값 높은 순대로 sub_category 출력

    # 확률값까지 보고 싶을 때
    for key, value in sorted_rank.items():
        print(sub_type_mapping_reverse.get(key), '  ', value)

In [ ]:
get_category_name('네비게이션이 안 돼요')

TypeError: ignored